# Collegarsi ad un database con Python

In questo notebook vedremo come è possibile collegarsi ad un database utilizzando Python.

Utilizzeremo come database [SQLite](https://sqlite.org/index.html), una libreria che consente di creare un database autocontenuto in un singolo file interrogabile con linguaggio SQL.

Per interfacciarci al database utilizzeremo SQLAlchemy, una delle librerie più usate in Python per interrogare i database.

Per eseguire una cella si può cliccare su Run presente in alto, oppure tasto shift + invio

### SQLAlchemy
[SQLAlchemy](https://www.sqlalchemy.org/) è una libreria di Python che consente di interfacciarsi con diversi database per recuperare dati da utilizzare all'interno di un'applicazione.

Per interrogare un database sono possibili due modalità (non solo per questa libreria, ma in generale con anche altri linguaggi):
 * **RAW Query**: si crea la base di dati manualmente e scrivono le interrogazioni in linguaggio SQL
 * **Object Relational Mapping**: si crea la base di dati sottoforma di una collezione di oggetti e ci si interfaccia adessa interagendo con gli oggetti, tutta la gestione è demandata alla libreria che si sta utilizzando.


Vediamo con un esempio come funzionano i due metodi.



Supponiamo di voler definire questo semplice schema:

&nbsp;&nbsp;&nbsp;studente (<u>sid</u>, cf, nome, cognome)\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **AK**: cf

&nbsp;&nbsp;&nbsp;corso (<u>cid</u>, nome)

&nbsp;&nbsp;&nbsp;esame (<u>data, sid, cid</u>, voto)\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **FK**: sid **REFERENCES** studente\
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; **FK**: cid **REFERENCES** corso

Iniziamo importando la librerie di SQLAlchemy.

Per visualizzare i risultati utilizzeremo Pandas, una libreria che consente di gestire dati tabellari (dataframe).

In [ ]:
import sqlalchemy
from sqlalchemy import text
import pandas as pd
from IPython.display import display
import os

# Modalità RAW
La modalità raw consente di eseguire istruzioni SQL pure, scrivendole direttamente.

In SQLAlchemy si utilizza la funzione [text](https://docs.sqlalchemy.org/en/14/core/sqlelement.html#sqlalchemy.sql.expression.text) per eseguire istruzioni SQL.

## Definire una connessione

Innanzi tutto creiamo un nuovo database chiamato test.db

Il database verrà creato nella stessa directory in cui è in esecuzione questo notebook

In [ ]:
# Constrolla se il database esiste, e se esiste lo elimina
if os.path.isfile('raw_db.db'):
  os.remove('raw_db.db')

# Crea il database
engine = sqlalchemy.create_engine('sqlite:///raw_db.db', echo=False)

# Apre la connessioneal database
conn = engine.connect()

SQL Alchemy consente la connessione a diversi DBMS, ad esempio una connessione a PostgreSQL può essere impostata nel seguente modo.

```
dbuser = "user"          # Nome utente
dbpass = "passwd"          # Password
dbname = "test_raw"        # Nome del database
dbhost = "127.0.0.1:5432"  # Host del DBMS e porta a cui risponde

# Definizione della connessione
engine = sqlalchemy.create_engine("postgresql://"+dbuser+":"+dbpass+"@"+dbhost+"/"+dbname)
```



## Creazione delle tabelle

In [ ]:
# Esegue le istruzioni SQL: crea le tabelle
conn.execute(text("""
    CREATE TABLE studente (
        sid INT PRIMARY KEY,
        cf CHAR(16) NOT NULL UNIQUE,
        nome VARCHAR(255) NOT NULL,
        cognome VARCHAR(255) NOT NULL
    );"""))

conn.execute(text("""
    CREATE TABLE corso (
        cid INT PRIMARY KEY,
        nome VARCHAR(255) NOT NULL
    );"""))

conn.execute(text("""
    CREATE TABLE esame (
        data DATE NOT NULL,
        sid INT NOT NULL REFERENCES studente(sid),
        cid INT NOT NULL REFERENCES corso(cid),
        voto INT NOT NULL,
        PRIMARY KEY(data, sid, cid)
    );
"""))

## Inserimento dei dati

Inserimento dei dati: inseriamo alcuni record

In [ ]:
# Apre la connessione
conn = engine.connect()

# Esegue le istruzioni SQL: crea le tabelle
conn.execute(text("""INSERT INTO studente(sid, cf, nome, cognome) VALUES (1, '12345', 'Mario', 'Rossi');"""))
conn.execute(text("""INSERT INTO corso(cid, nome) VALUES (1, 'Basi di dati');"""))
conn.execute(text("""INSERT INTO esame(data, sid, cid, voto) VALUES ('06/06/2021', 1, 1, 25);"""))
conn.commit()

# Chiude la connessione
conn.close()

## Lettura dei dati
Possiamo leggere i dati dal database:

In [ ]:
# Apre la connessione
conn = engine.connect()

# E' possibile parametrizzare una query in questo modo
t = text("SELECT * FROM studente WHERE nome=:nome")
results = conn.execute(t, {"nome":'Mario'})

# Leggo i risultati
df = pd.DataFrame(results)

# Chiude la connessione
conn.close()

df

# Modalità Object Relational Mapping (ORM)
L'ORM è una tecnica di programmazione che astrae il livello fisico del DBMS e rappresenta le tabelle sottoforma di classi, ogni record inserito in una tabella viene rappresentato come un oggetto.

In questo modo è il software a gestire la creazione del database e non ci si deve preoccupare della portabilità se si cambia il DBMS.

In [ ]:
from sqlalchemy import Column, ForeignKey, Integer, String, Date, PrimaryKeyConstraint, select
from sqlalchemy.orm import declarative_base, relationship, Session

## Creare il database e definire una connessione

Crea il database e crea la connessione

In [ ]:
# Constrolla se il database esiste, e se esiste lo elimina
if os.path.isfile('test_orm.db'):
  os.remove('test_orm.db')

# Crea il database
engine = sqlalchemy.create_engine('sqlite:///test_orm.db', echo=False)

# Apre la connessioneal database
conn = engine.connect()

## Creazione delle tabelle
Innanzi tutto bisogna definire la classe base che SQLAlchemy usa per definire gli oggetti che rappresentano le tabelle

In [ ]:
Base = declarative_base()

Ogni tabella deve essere definita come una classe.
SQLAlchemy offre vari [tipi di dato](https://docs.sqlalchemy.org/en/14/core/type_basics.html) e la possibilità di esprimere tutti i [vincoli presenti in SQL](https://docs.sqlalchemy.org/en/14/core/constraints.html).

In [ ]:
# Tabella Studente
class Studente(Base):
    __tablename__ = "studente"
    sid = Column(Integer, primary_key=True)
    # Definisce il vincolo di AK
    cf = Column(String(16), nullable=False, unique=True)
    nome = Column(String, nullable=False)
    cognome = Column(String, nullable=False)

    def __repr__(self):
        """
        Metodo utilizzato quando si vuole visualizzare un oggetto di tipo studente
        """
        return f"sid={self.sid}, cf={self.cf}, nome={self.nome}, cognome={self.cognome}"

class Corso(Base):
    __tablename__ = "corso"
    cid = Column(Integer, primary_key=True)
    nome = Column(String, nullable=False)

    def __repr__(self):
        return f"cid={self.cid}, nome={self.nome}"

class Esame(Base):
    __tablename__ = "esame"
    data = Column(Date, nullable=False)
    # Definisce anche il vincolo di FK
    sid = Column(Integer, ForeignKey("studente.sid"), nullable=False)
    cid = Column(Integer, ForeignKey("corso.cid"), nullable=False)
    voto = Column(Integer, nullable=False)

    # Definise la primary key su più attributi
    PrimaryKeyConstraint(data, sid, cid)

    def __repr__(self):
        return f"data={self.data}, sid={self.sid}, cid={self.cid}, voto={self.voto}"


Una volta create le tabelle, queste devono essere scritte sul database eseguendo la seguente istruzione

In [ ]:
Base.metadata.create_all(engine)

## Inserimento dei dati

Innanzi tutto bisogna stabilire una nuova sessione con il database

In [ ]:
session = Session(engine)

Poi si possono creare i record, ogni record è un'istanza della classe in cui deve essere inserito

In [ ]:
from datetime import datetime

s1 = Studente(
    sid = 1,
    cf = "123456",
    nome = "Mario",
    cognome = "Rossi"
)
c1 = Corso(
    cid = 1,
    nome = "Basi di dati"
)
e1 = Esame(
    data = datetime.strptime("06/06/2021", '%d/%m/%Y'),
    sid = 1,
    cid = 1,
    voto = 25
)

Per inserire un record bisogna aggiungerlo alla sessione corrente e poi chiamare l'istruzione `commit` per scrivere i dati sul database.

In [ ]:
session.add_all([s1, c1])
session.commit()

session.add(e1)
session.commit()

## Interrogare il database
Le query devono essere composte con l'istruzione `select`, SQLAlchemy offre vari [metodi](https://docs.sqlalchemy.org/en/14/core/selectable.html) per creare query avanzate

In [ ]:
query = select(Studente).where(Studente.cognome == "Rossi")

Il risultato di una query è un elenco di oggetti della classe che definisce la tabella

In [ ]:
for studente in session.execute(query):
    print(studente[0])

Esempio di query con JOIN che restituisce più oggetti, ogni record è un oggetto della classe contenuta nella select

In [ ]:
query = select(Studente, Esame, Corso)\
       .join_from(Studente, Esame, Studente.sid == Esame.sid)\
       .join_from(Esame, Corso, Esame.cid == Corso.cid)

In [ ]:
for row in session.execute(query):
    print(f"{row[0].nome} {row[0].cognome}")
    print(f"{row[2].nome}")
    print(f"{row[1].data} - {row[1].voto}")


#RAW o ORM?
### Vantaggi di utilizzare un sistema ORM

* **Velocizza lo sviluppo e ne riduce i costi**: non è necessario progettare il database e tutte le funzioni per convertire il risultato di una query in un formato utilizzabile dall'applicazione;
* **Supera le differenze tra i vari vendor**: il motore ORM sa come scrivere query specifiche per ogni DBMS e quindi non bisogna preoccuparsene;
* **Protezione**: prevede controlli sugli input per prevenire l'SQL injection;
* **Funzioni avanzate**: può fornire facilmente funzioni avanzate come supporto per transazioni, etc.

### Svantaggi

* **Complessità**: il sistema che traduce le query è molto complesso, quindi risulta essere inefficiente rispetto a un sistema di interrogazione diretta del database;
* **Difficoltà nel debug**: non è facile eseguire il debug perché non si riesce a vedere cosa fa internamente il sistema ORM e come genera le query;
* **Apprendimento lento**: uno sviluppatore deve studiare ed imparare ad utilizzare il sistema ORM anche se conosce già l'SQL;
* **Non si può fare tutto**: query particolarmente complesse non si riescono a fare con ORM e bisogna ricorrere alla modalità RAW.

